In [1]:
from gensim.test.utils import datapath
from gensim.models import FastText
import pandas as pd
import nltk
from gensim.parsing.preprocessing import STOPWORDS
import re
import spacy
import numpy as np
import tqdm 
import pandas as pd
import tqdm
nltk.download('stopwords') 
nltk.download('wordnet') 

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
STOPWORDS

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [4]:
data_tip=pd.read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json', lines=True,)[:700]


In [5]:
len(data_tip)

700

In [6]:
tip_text = data_tip['text']

In [7]:
tip_text[:3]

0                       Avengers time with the ladies.
1    They have lots of good deserts and tasty cuban...
2               It's open even when you think it isn't
Name: text, dtype: object

In [8]:
import spacy

def process_text(document,stopwords=STOPWORDS):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    doc_spacy=nlp(document)
    lemma_txt = ["".join(token.lemma_) for token in tqdm.tqdm(doc_spacy)]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in STOPWORDS]
    lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return lemma_no_stop_txt

tip_text_preprocessed = tip_text.apply(process_text)



100%|██████████| 15/15 [00:00<00:00, 83110.38it/s]


In [9]:
tip_text_preprocessed

0                                  [avenger, time, lady]
1                 [good, desert, tasty, cuban, sandwich]
2                                          [open, think]
3                               [decent, fried, chicken]
4                   [appetizer, platter, special, lunch]
                             ...                        
695       [great, restaurant, great, food, fresh, order]
696                                    [seating, squish]
697      [like, especially, feel, hungry, cheese, taste]
698                     [forget, close, roasted, potato]
699    [wing, eating, contest, swing, come, yelp, shirt]
Name: text, Length: 700, dtype: object

In [10]:
model3= FastText(vector_size=312,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)

In [11]:
model3.wv.vectors_vocab.shape 

(1448, 312)

In [12]:
model3.ns_exponent #PROBABILITY OF NEGATIVE WORD SAMPLING

0.75

In [13]:
test_words="Pizza Burger Coffee Restaurant Delivery Review Menu Rating Service Ambience".split()
test_words=list(np.array(test_words).astype(str))

In [14]:
def test_words_andmodel_updating(model, test_words):
    similar_words = []
    un_similar_words=[]
    embedding_all_test_words_list_separable=[]
    for word in test_words:
        similar_word = model.wv.most_similar(positive=[word], topn=10)
        unsimilar_word =model.wv.most_similar(negative=[word], topn=10)
        embedding_each_test_word=model.wv[word]
        embedding_all_test_words_list_separable.append(embedding_each_test_word)
        similar_words.append(similar_word)
        un_similar_words.append(unsimilar_word)
        model3= FastText(vector_size=312,workers=20,window=3,sg=1, min_count=1,sentences=tip_text_preprocessed, epochs=1000)
    return pd.DataFrame({"test_embedding":embedding_all_test_words_list_separable,"most_similar":similar_words , "most_opposite":un_similar_words})


In [15]:
test_custom_trained_model_updating=test_words_andmodel_updating(model3,test_words)

In [16]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
test_custom_trained_model_updating.to_csv('/kaggle/working/my_trained_model_test__.csv', index=False)


In [17]:
test_custom_trained_model_updating

,test_embedding,most_similar,most_opposite
0,"[-0.1737498, 0.19779637, -0.055288944, -0.060962703, -0.22139193, 0.010335644, 0.07600514, 0.29671103, -0.21763977, -0.051683616, 0.15158512, 0.2039709, 0.044685993, 0.05953786, -0.03249718, 0.15784323, 0.25804934, 0.047876887, -0.13081434, 0.110151954, 0.120611034, 0.013077468, -0.23058775, 0.13920096, -0.24774016, -0.12074392, 0.031475056, -0.1622686, -0.07780725, -0.01900448, 0.0059634107, 0.09179949, -0.024754943, -0.008380892, -0.116043344, 0.13914588, 0.13013399, 0.10694631, -0.087851174, 0.012115532, -0.0762545, 0.24398604, 0.036666993, 0.0831665, 0.26692337, -0.26521233, -0.09163118, -0.12670447, -0.23744914, -0.1639758, 0.052508075, -0.097854786, 0.06798993, 0.24555056, -0.08628303, 0.2599579, -0.18500371, 0.09351752, -0.0072564175, -0.034437332, 0.10737087, 0.20681657, 0.13997339, -0.14065148, -0.20074354, 0.10144983, -0.24665956, 0.50276965, 0.059985407, 0.12630747, -0.026124734, 0.07677581, 0.12127585, -0.14169866, -0.035376776, -0.0054034884, 0.223032, 0.03216798, -0.19025077, -0.15493837, 0.062071588, -0.223523, 0.008533849, 0.2992347, 0.062763564, 0.13944949, 0.0009407144, 0.36195606, 0.08363537, 0.1679736, -0.06114157, 0.05861833, 0.017478062, 0.19288465, 0.025802612, -0.014593604, -0.31473878, -0.037167974, -0.09573419, 0.003738187, ...]","[(pizza, 0.9906517863273621), (sonoran, 0.7599024772644043), (bruschetta, 0.7366751432418823), (hebert, 0.7304633855819702), (pint, 0.7133648991584778), (vegan, 0.7131001353263855), (irvington, 0.6776270270347595), (anytime, 0.6753858923912048), (church, 0.6635223031044006), (word, 0.6569406986236572)]","[(able, -0.022981900721788406), (available, -0.041228871792554855), (edible, -0.057957280427217484), (hole, -0.060657888650894165), (table, -0.06439768522977829), (remarkable, -0.06470245122909546), (free, -0.08411397784948349), (wall, -0.09022192656993866), (tree, -0.09755486994981766), (character, -0.0977519303560257)]"
1,"[-0.28783733, 0.35119125, -0.38681778, -0.20309404, -0.112580284, 0.3795525, 0.47134936, -0.38025057, 0.085835606, -0.27016768, 0.45441613, 0.25647023, 0.28182867, 0.26618502, -0.011909936, 0.3125005, -0.27107212, -0.027635911, -0.23122141, 0.4019272, -0.05322136, -0.11195914, -0.23477061, 0.041014902, 0.0569802, -0.14268522, 0.19246224, -0.016687606, -0.7301532, 0.01519739, -0.1546017, 0.064032346, 0.2880976, -0.023367323, 0.12385331, 0.6408261, -0.00127746, -0.21909875, 0.26569098, -0.037898857, -0.3611972, 0.105413884, 0.031925082, -0.27984035, 0.23455125, -0.3107682, -0.06637679, 0.07555946, -0.40837166, -0.10974638, 0.11702291, -0.16609976, 0.2109655, 0.33526936, -0.12826602, -0.20270082, 0.18430874, -0.121195756, -0.27647507, 0.19898616, -0.3013, 0.34396213, -0.07804748, -0.18642735, 0.08538103, -0.24947432, -0.29847968, -0.11118984, 0.30145097, -0.15060446, 0.3250594, -0.23775148, -0.0019467539, -0.06561467, 0.010109897, -0.2823028, -0.49848747, -0.3302776, -0.46122825, 0.24639368, 0.026563657, -0.1647612, -0.32008287, -0.080581754, -0.14771217, 0.15512499, -0.067083105, 0.22572698, 0.3248335, -0.025945332, -0.13648896, -0.16402449, 0.06193222, 0.24235207, 0.40327245, -0.502143, -0.6231792, 0.2249936, -0.44925806, 0.09195236, ...]","[(burger, 0.9458807706832886), (hamburger, 0.8435412049293518), (cheeseburger, 0.7783929109573364), (buger, 0.7671096324920654), (avenger, 0.7271824479103088), (banger, 0.7157685160636902), (manager, 0.702721118927002), (lawyer, 0.6996210813522339), (pacer, 0.680546760559082), (cider, 0.6381554007530212)]","[(scan, -0.14808547496795654), (family, -0.15278016030788422), (brazilian, -0.1621040552854538), (stink, -0.16318728029727936), (inside, -0.1649225652217865), (high, -0.16560731828212738), (mills, -0.16660752892494202), (breakfast, -0.16899831593036652), (opry, -0.16994315385818481), (plus, -0.17093688249588013)]"
2,"[-0.1814742, 0.23781154, -0.1517391, 0.14001673, 0.2364192, 0.067154534, -0.07848684, 0.1674487, -0.13792832, -0.2233023, 0.32992566, 0.06700814, 0.03694498

In [18]:
np.array(test_custom_trained_model_updating['test_embedding'][0]).shape

(312,)

# Facebook pretrained model

In [19]:
!pip install fasttext

In [ ]:
! pip install hf_hub_download

In [20]:
import fasttext
from huggingface_hub import hf_hub_download

model_path = hf_hub_download(repo_id="facebook/fasttext-et-vectors", filename="model.bin")
model = fasttext.load_model(model_path)
model.words

model.bin:   0%|          | 0.00/7.24G [00:00<?, ?B/s]

[',',
 '.',
 '</s>',
 'ja',
 '"',
 'on',
 ')',
 '(',
 ':',
 "'",
 '–',
 'et',
 'ei',
 'oli',
 'ka',
 'ning',
 '/',
 'kui',
 'Eesti',
 'mis',
 'või',
 'ta',
 '-',
 'oma',
 'aastal',
 ';',
 '#',
 'see',
 'aasta',
 '!',
 '?',
 'siis',
 'aga',
 'kes',
 '1',
 'selle',
 'seda',
 'nii',
 'võib',
 'välja',
 'kuid',
 'pole',
 '%',
 'kus',
 'sai',
 '2',
 'Ta',
 'mille',
 'ole',
 'mida',
 'üle',
 'Kui',
 '“',
 'tema',
 'eesti',
 'veel',
 '_',
 'kuni',
 '|',
 'saab',
 'Tartu',
 'See',
 'ehk',
 '€',
 'Tallinna',
 'nende',
 'üks',
 '3',
 'väga',
 'kohta',
 'järgi',
 'keeles',
 '}',
 '„',
 '...',
 'eest',
 'juba',
 'nagu',
 '”',
 'ajal',
 'olid',
 'pärast',
 'i',
 'koos',
 'vaid',
 'kõik',
 'tuleb',
 'vastu',
 'olla',
 'ma',
 '4',
 'Euroopa',
 '10',
 'of',
 'nad',
 'sest',
 'osa',
 'olnud',
 'poolt',
 'pisi',
 'palju',
 'mitte',
 'Kategooria',
 'näiteks',
 'USA',
 'kõige',
 'alates',
 'läbi',
 'Eestis',
 '5',
 'kas',
 'ainult',
 'teha',
 'ära',
 'kaks',
 '•',
 'I',
 'neid',
 'meie',
 'need',
 'tagasi

In [21]:
def test_model_basedon_facebook_pretrained_model(model_1,model_2,test_words):
    check_list=[]
    for word in test_words:
        check_list.append(np.allclose(model_1.wv[word],model_2.wv[word]))
    return check_list


In [22]:
test_model_basedon_facebook_pretrained_model(model_PRETRAINED,model3,test_words)

NameError: name 'model_PRETRAINED' is not defined

In [2]:
import fasttext